In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
client.create_experiment(name='my-cool-experiment') # para crear un nuevo experimento

'2'

In [11]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids= '1', # id del experimento
    filter_string="metrics.rmse < 6.5",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d46a9372875f43dc93bc417a5addfef2, rmse: 6.3643
run id: 5b83a6a970c949c48fdfd5e381a212cb, rmse: 6.3643
run id: bc1848cadda0406e9456c532f8f7f072, rmse: 6.3643
run id: c1c83c48e46e447f9b3c3f2c1f6c636a, rmse: 6.4843


# Model registry

In [13]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [14]:
# Registrar un nuevo modelo

run_id = 'c1c83c48e46e447f9b3c3f2c1f6c636a'
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1717278122639, current_stage='None', description=None, last_updated_timestamp=1717278122639, name='nyc-taxi-regressor', run_id='c1c83c48e46e447f9b3c3f2c1f6c636a', run_link=None, source='file:///c:/Users/daniel/Documents/Universidad/Cursos/MLOpsZoomcamp/02_experiment-tracking/mlruns/1/c1c83c48e46e447f9b3c3f2c1f6c636a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [18]:
# Listar versiones de un modelo registrado y su etapa actual

model_name = 'nyc-taxi-regressor'

latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production
version: 3, stage: None


C:\Users\daniel\AppData\Local\Temp\ipykernel_1340\840208094.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [21]:
# Transicionar una versión de un modelo a una etapa específica
model_version = 3
new_stage = "Staging"


client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Staging",
    archive_existing_versions=False
)

C:\Users\daniel\AppData\Local\Temp\ipykernel_1340\3424978932.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717278122639, current_stage='Staging', description=None, last_updated_timestamp=1717279091769, name='nyc-taxi-regressor', run_id='c1c83c48e46e447f9b3c3f2c1f6c636a', run_link=None, source='file:///c:/Users/daniel/Documents/Universidad/Cursos/MLOpsZoomcamp/02_experiment-tracking/mlruns/1/c1c83c48e46e447f9b3c3f2c1f6c636a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [22]:
# Actualizar la descripción de una versión de un modelo

from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {version} was transitioned to {new_stage} stage on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1717278122639, current_stage='Staging', description=('The model version <ModelVersion: aliases=[], '
 "creation_timestamp=1717278122639, current_stage='None', description=None, "
 "last_updated_timestamp=1717278122639, name='nyc-taxi-regressor', "
 "run_id='c1c83c48e46e447f9b3c3f2c1f6c636a', run_link=None, "
 "source='file:///c:/Users/daniel/Documents/Universidad/Cursos/MLOpsZoomcamp/02_experiment-tracking/mlruns/1/c1c83c48e46e447f9b3c3f2c1f6c636a/artifacts/model', "
 "status='READY', status_message=None, tags={}, user_id=None, version=3> was "
 'transitioned to Staging stage on 2024-06-01'), last_updated_timestamp=1717279093783, name='nyc-taxi-regressor', run_id='c1c83c48e46e447f9b3c3f2c1f6c636a', run_link=None, source='file:///c:/Users/daniel/Documents/Universidad/Cursos/MLOpsZoomcamp/02_experiment-tracking/mlruns/1/c1c83c48e46e447f9b3c3f2c1f6c636a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [23]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}



In [24]:
df = read_dataframe('data/green_tripdata_2021-03.parquet')

In [27]:
run_id = 'd46a9372875f43dc93bc417a5addfef2'
client.download_artifacts(run_id = run_id, path = 'preprocessor', dst_path='.') # descargar el preprocesador

'c:\\Users\\daniel\\Documents\\Universidad\\Cursos\\MLOpsZoomcamp\\02_experiment-tracking\\preprocessor'

In [28]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [29]:
X_test = preprocess(df, dv)

In [30]:
target = 'duration'
y_test = df[target]

In [31]:
%time test_model(name= model_name, stage='Production',X_test= X_test,y_test= y_test)

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [16:20:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 8.92 s
Wall time: 6.8 s


{'rmse': 6.300736074421472}

In [33]:
%time test_model(name= model_name, stage='Staging',X_test= X_test,y_test= y_test)

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [16:22:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 5.56 s
Wall time: 916 ms


{'rmse': 6.300736074421472}

In [34]:
model_version = 1
new_stage = "Production"


client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

C:\Users\daniel\AppData\Local\Temp\ipykernel_1340\3637187561.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717276914317, current_stage='Production', description='', last_updated_timestamp=1717280765300, name='nyc-taxi-regressor', run_id='d46a9372875f43dc93bc417a5addfef2', run_link='', source='file:///c:/Users/daniel/Documents/Universidad/Cursos/MLOpsZoomcamp/02_experiment-tracking/mlruns/1/d46a9372875f43dc93bc417a5addfef2/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>